# Skyrmion U-Net prediction & analysis example

## Import packages + define utility functions

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import cv2
import scipy.spatial
import glob
import io
import pandas as pd
import ipywidgets
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use('seaborn-v0_8-dark')

In [ ]:
gpu_available = lambda : len(tf.config.list_physical_devices('GPU'))
if gpu_available(): print("GPU available")

# U-Net architecture
The U-Net models in this repository were trained to predict on 512x512 Kerr microscopy images.




![](notebook_figures/u_net_architecture_1.png)

In [ ]:
# Basic activation layer
class MishLayer(tf.keras.layers.Layer):
    def call(self, x):
        return tf.keras.activations.mish(x)

# Basic Convolution Block
def conv_block(x, n_channels, param):
    x = tf.keras.layers.Conv2D(n_channels, kernel_size=param["kernel_size"],kernel_initializer=param["kernel_initialization"],padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x) 
    x = MishLayer()(x)
    return x

# Double Convolution Block used in "encoder" and "bottleneck"
def double_conv_block(x, n_channels, param):
    x = conv_block(x,n_channels,param)
    x = conv_block(x,n_channels,param)
    return x

# Downsample block for feature extraction (encoder)
def downsample_block(x, n_channels, param):
    f = double_conv_block(x, n_channels, param)
    p = tf.keras.layers.MaxPool2D(pool_size=(2,2))(f)
    p = tf.keras.layers.Dropout(param["dropout"])(p)
    return f, p

# Upsample block for the decoder
def upsample_block(x, conv_features, n_channels, param):
    x = tf.keras.layers.Conv2DTranspose(n_channels*param["upsample_channel_multiplier"], param["kernel_size"], strides=(2,2), padding='same')(x)
    x = tf.keras.layers.concatenate([x, conv_features])
    x = tf.keras.layers.Dropout(param["dropout"])(x)
    x = double_conv_block(x, n_channels, param)
    return x

# Create the model
def get_unet(param):
    input = tf.keras.layers.Input(shape=param["input_shape"]+(1,))
    next_input = input
    
    l_residual_con = []
    for i in range(param["n_depth"]):
        residual_con,next_input = downsample_block(next_input, (2**i)*param["filter_multiplier"],param)
        l_residual_con.append(residual_con)

    next_input = double_conv_block(next_input, (2**param["n_depth"])*param["filter_multiplier"],param)

    for i in range(param["n_depth"]):
        next_input = upsample_block(next_input, l_residual_con[param["n_depth"]-1-i], (2**(param["n_depth"]-1-i))*param["filter_multiplier"],param)

    output = tf.keras.layers.Conv2D(param["n_class"], (1,1), padding="same", activation = "softmax",dtype='float32')(next_input)    
    
    return tf.keras.Model(input, output, name=param["name"])

U-Net can be described as follows:

$$ \mathbf{z} = \mathbf{f}_{\vec{\theta}}(\mathbf{x}) $$

where $\vec{\theta}$ is a high-dimensional vector containing all parameters, $\mathbf{z}$ is a 3-dimensional output tensor, and $\mathbf{x}$ is the input Kerr microscopy image. In $\mathbf{f}$ are encapsulated all the operations such as convolution, max pooling, up-convolution, batch normalization, ... .

$\mathbf{z}$ is converted to probabilities using softmax:

$$ p_{(x,y),i} = \frac{\exp(\mathbf{z}_{(x,y),i})}{\sum_{i\in \mathrm{classes}} \exp(\mathbf{z}_{(x,y),i})} $$

The output mask $\mathbf{m}$ is then determined by:

$$ \mathbf{m}_{(x,y)} = \mathrm{arg\,max}_i\; p_{\mathrm{U-Net}}(\mathbf{z}_{(x,y),i}). $$

# Mask index $\leftrightarrow$ RGB image

### Comment

Labels for U-Net model:
- Skyrmions (RGB-label: red (255,0,0))
- Defects (RGB-label: green (0,255,0)
- FM background (RGB-label: blue (0,0,255))
- Non-FM background (RGB-label: yellow (255,255,0))
- Boundary non-FM/FM background (RGB-label: cyan (0,255,255))

The 3-class U-Net model predicts skyrmions (RGB-label: red (255,0,0)), defects (RGB-label: blue (0,255,0)), and background (RGB-label: blue (0,0,255)). The background consists of the ferromagnetic background, non-ferromagnetic background, and the boundary between ferromagnetic and non-ferromagnetic background. The RGB-label is converted to an class index for training. For model 2022 the class indeces are (skyrmions:0, defects:1, background:2), for model 2023 the class indeces are (skyrmions:0, background:1, defects:2)

In [ ]:
def trafo_channel_to_rgb(I):
    basis = np.array([[255,0,0],[0,255,0],[0,0,255]],dtype=np.uint8)
    return basis[I]

def trafo_rgb_to_channel(I):
    Q = np.zeros((I.shape[0],I.shape[1]),dtype=np.uint8)
    R,G,B = I[:,:,0],I[:,:,1],I[:,:,2]
    skyrmion_mask = (R>=128)&(G<128)&(B<128)
    defect_mask = (R<128)&(G>=128)&(B<128)
    bck_mask = ~(skyrmion_mask|defect_mask)
    Q[skyrmion_mask] = 0
    Q[defect_mask] = 1
    Q[bck_mask] = 2
    return Q

# Prediction with U-Net

In [ ]:
#Predict the label based on Kerr images and the U-Net model.
def predict(x,fn_model,batch_size=5,normalize_255=True):
    #load U-Net model
    model = tf.keras.models.load_model(fn_model,compile=False,custom_objects={'MishLayer': MishLayer})
    
    if not gpu_available():
        #create identical model, only with pure float_32 policy
        batch_size = 1
        nmodel = get_unet({"name":"unet","input_shape": (512,512), "n_class":3,"filter_multiplier":16,"n_depth":4,
                  "kernel_initialization":"he_normal","dropout":0.1,"kernel_size":(3,3),"upsample_channel_multiplier":8})
        nmodel.set_weights(model.weights)
        model = nmodel
    
    n = int(np.ceil(len(x)/batch_size))
    lix = [np.array(range(j*batch_size,min((j+1)*batch_size,len(x)))) for j in range(n)]
    ylabel = np.zeros(x.shape,dtype=np.uint8)
    progbar = tf.keras.utils.Progbar(n)
    for i in range(n):            
        progbar.update(i)
        input = x[lix[i]]
        if normalize_255:
            input = input/255
        ylabel[lix[i]] = model.predict(input,verbose=False).argmax(-1)
    progbar.update(n,finalize=True)
    return ylabel

# Validation - Matthews correlation coefficient (MCC)
$$\mathrm{MCC} = \frac{\mathrm{TP} \cdot \mathrm{TN} - \mathrm{FP} \cdot \mathrm{FN}}{\sqrt{(\mathrm{TP} + \mathrm{FP})(\mathrm{FN} + \mathrm{TN})(\mathrm{TP} + \mathrm{FN})(\mathrm{FP} + \mathrm{TN})}}$$
with the number of true positives $\mathrm{TP}$, true negatives $\mathrm{TN}$, false positives $\mathrm{FP}$, and false negatives $\mathrm{FN}$.

In [ ]:
def get_TF_PN(y_true,y_pred,ix0):
    m1,m2 = y_true==ix0,y_pred==ix0
    im1,im2 = tf.math.logical_not(m1),tf.math.logical_not(m2)
    TP = tf.math.reduce_mean(tf.cast(tf.math.logical_and(m1,m2),dtype=np.float64))
    TN = tf.math.reduce_mean(tf.cast(tf.math.logical_and(im1,im2),dtype=np.float64))
    FP = tf.math.reduce_mean(tf.cast(tf.math.logical_and(im1,m2),dtype=np.float64))
    FN = tf.math.reduce_mean(tf.cast(tf.math.logical_and(m1,im2),dtype=np.float64))
    return TP,TN,FP,FN

def get_mcc_from_TF_PN(TP,TN,FP,FN):
    denom = tf.keras.ops.sqrt((TP + FN) * (FP + TN) * (FP + TP) * (FN + TN))
    val = (TP * TN - FP * FN) / denom
    return  tf.where(tf.equal(denom, 0), tf.constant(0, dtype=tf.float64), val)

def get_mcc(y_true,y_false,n_class):
    return get_mcc_from_TF_PN(*get_TF_PN(y_true,y_false,n_class)).numpy()

# Prediction of Kerr microscopy images from dataset

## Load filenames of dataset

In [ ]:
dataset = pd.read_csv("dataset/table.csv",sep=";")
fnimg,fnlabel = list(dataset.img_fn.to_numpy()),list(dataset.label_fn.to_numpy())
dataset

## Model 2023

In [ ]:
#load imag and label
ix = 129
img = np.array(Image.open(fnimg[ix]))
label = np.array(Image.open(fnlabel[ix]))
#cut to 512x512
img,label = img[:512,:512],label[:512,:512]
#Predict label with U-Net
predicted_label = predict(np.array([img]),"models/2023_model.keras",batch_size=1)[0]
#Swap class index of 1 and 2, since for model 2023 the class indeces are (skyrmion:0, background:1, defects:2) and the functions are written for class indeces (skyrmion:0, defects:1, background:2)
predicted_label[predicted_label==1] = 5
predicted_label[predicted_label==2] = 1
predicted_label[predicted_label==5] = 2
#Evaluate predicitionn with Matthews correlation coefficient
print("Pixelwise Matthews correlation coefficient (true=skyrmion,false=defect,background)",get_mcc(trafo_rgb_to_channel(label),predicted_label,0))

In [ ]:
plt.close("all")
%matplotlib inline

fig,ax = plt.subplots(ncols=3,dpi=300)
ax[0].imshow(img,cmap="gray")
ax[1].imshow(label)
ax[2].imshow(trafo_channel_to_rgb(predicted_label))
ax[0].set_title("Kerr image")
ax[1].set_title("Ground truth")
ax[2].set_title("Predicted label")
fig.tight_layout()

## Model 2022

In [ ]:
#load imag and label
ix = 78
img = np.array(Image.open(fnimg[ix]))
label = np.array(Image.open(fnlabel[ix]))
#cut to 512x512
img,label = img[:512,:512],label[:512,:512]
#Predict label with U-Net
predicted_label = predict(np.array([img]),"models/2022_model.keras",batch_size=1)[0]
#Evaluate predicitionn with Matthews correlation coefficient
print("Pixelwise Matthews correlation coefficient (true=skyrmion,false=defect,background)",get_mcc(trafo_rgb_to_channel(label),predicted_label,0))

In [ ]:
plt.close("all")
%matplotlib inline

fig,ax = plt.subplots(ncols=3,dpi=300)
ax[0].imshow(img,cmap="gray")
ax[1].imshow(label)
ax[2].imshow(trafo_channel_to_rgb(predicted_label))
ax[0].set_title("Kerr image")
ax[1].set_title("Ground truth")
ax[2].set_title("Predicted label")
fig.tight_layout()

## Model Inversion 2022
Model that also functions on Kerr microscopy images with normal and inverted intensity.

In [ ]:
#load imag and label
ix = 1
img = 255-np.array(Image.open(fnimg[ix]))
label = np.array(Image.open(fnlabel[ix]))
#cut to 512x512
img,label = img[:512,:512],label[:512,:512]
#Predict label with U-Net
predicted_label = predict(np.array([img]),"models/2022_model_inv.keras",batch_size=1)[0]
#Evaluate predicitionn with Matthews correlation coefficient
print("Pixelwise Matthews correlation coefficient (true=skyrmion,false=defect,background)",get_mcc(trafo_rgb_to_channel(label),predicted_label,0))

In [ ]:
plt.close("all")
%matplotlib inline

fig,ax = plt.subplots(ncols=3,dpi=300)
ax[0].imshow(img,cmap="gray")
ax[1].imshow(label)
ax[2].imshow(trafo_channel_to_rgb(predicted_label))
ax[0].set_title("Kerr image")
ax[1].set_title("Ground truth")
ax[2].set_title("Predicted label")
fig.tight_layout()

# Skyrmion U-Net prediction & anaylsis of a (own) Kerr microscopy image

Example Kerr microscopy image for this section courtesy of Dr. E. Martin Jefremovas.

## Load & edit Kerr microscopy image

In [ ]:
%matplotlib widget
def manip(config):
    invert = config["inv"]
    x0,x1 = config["xc"]
    y0,y1 = config["yc"]
    img = config["input_img"].copy()
    if len(img.shape)>2:
        img =  np.mean(img,axis=-1)
        
    img = img[int(np.floor(y0)):int(np.ceil(y1)),int(np.floor(x0)):int(np.ceil(x1))]
    img = (img-np.min(img))/(np.max(img)-np.min(img))
    
    if invert:
        img = 1-img
    v0,v1 = config["intensity_clip"]
    nimg = np.clip(img,v0,v1)
    config["edited_img"] = np.clip(1/(v1-v0)*(img-v0),0,1)
    config["histinfo"] = img
    config["show_img"] = nimg


input_img = np.array(Image.open("example_kerr_microscopy_image.png"))
config_init = {"inv":False,"intensity_clip":(0,1),"xc":(0,input_img.shape[1]),"yc":(0,input_img.shape[0]),"input_img":input_img}
config = config_init.copy()

with plt.ioff():
    fig1,ax1 = plt.subplots(dpi=100)
    fig2,ax2 = plt.subplots(dpi=100,figsize=(2,2))
    fig1.canvas.header_visible = False
    fig2.canvas.header_visible = False
    
ax1.set_xlabel("x")
ax1.set_ylabel("y")
ax1.grid(False)
cmg = matplotlib.colormaps.get_cmap("gray")
clis = cmg(np.arange(cmg.N))
clis[0] = [1,0,0,1]
clis[-1] = [0,0,1,1]
ncmap = matplotlib.colors.ListedColormap(clis)
emptyimg = np.zeros((20,20)) #for initalisation of code only
im1 = ax1.imshow(emptyimg,cmap=ncmap,vmin=0,vmax=1)
cax = make_axes_locatable(ax1).append_axes("right",size = "2%",pad=0.03)
colb = plt.colorbar(im1,cax=cax)
bins = np.linspace(0,1,40)
_,_,hist = ax2.hist(emptyimg.flatten(),bins=bins,density=True)
ax2.set_xlim(0,1)
lowerl = ax2.axvline(0,color="red")
higherl = ax2.axvline(1,color="blue")
ax2.set_xlabel("Intensity")
fig2.tight_layout()

def update_1(canvasdraw=True):
    manip(config)
    img = config["show_img"]
    histinfo = config["histinfo"]
    
    im1.set_data(img)
    im1.set_extent((config["xc"][0],config["xc"][1],config["yc"][1],config["yc"][0]))
    
    lowerl.set_xdata([config["intensity_clip"][0],config["intensity_clip"][0]])
    higherl.set_xdata([config["intensity_clip"][1],config["intensity_clip"][1]])
    ax1.set_aspect(1)
    
    v0,v1 = config["intensity_clip"]
    im1.set_clim(v0,v1)
    colb.update_normal(im1)
    #hist.set_data(config["input_img"].copy().flatten())
    nhist = np.histogram(histinfo,bins,density=True)[0]
    for i,ele in enumerate(hist):
        ele.set_height(nhist[i])
    ax2.set_ylim(0,np.max(nhist)*1.1)
    
    if canvasdraw:
        fig1.canvas.draw()
        fig2.canvas.draw()
    
def update_img(inital_img):
    config["input_img"] = inital_img
    resetconfig()
    
def widget_change_a(a):
    config["a"]=a.new
    update_1()
def widget_change_b(b):
    config["b"]=b.new
    update_1()
def widget_change_x(b):
    config["xc"]=b.new
    update_1()
def widget_change_y(b):
    config["yc"]=b.new
    update_1()
def widget_change_intensity_clip(b):
    config["intensity_clip"]=b.new
    update_1()
def widget_change_rg(b):
    config["showclipping"]=b.new
    update_1()
def widget_change_inv(c):
    config["inv"]=c.new
    update_1()
def wreset():
    widget_unobserve()
    slider3.min = 0
    slider3.max = config["input_img"].shape[0]
    slider4.min = 0
    slider4.max = config["input_img"].shape[1]
    slider1.value = config["intensity_clip"]
    slider3.value = config["yc"]
    slider4.value = config["xc"]
    check1.value = config["inv"]
    widget_observe()

def resetconfig():
    config["xc"] = (0,config["input_img"].shape[1])
    config["yc"] = (0,config["input_img"].shape[0])
    for ele in ["inv","intensity_clip"]:
        config[ele] = config_init[ele] 
    wreset()
    update_1()

def widget_reset(b):
    resetconfig()
    
def updatefile(val):
    update_img(np.array(Image.open(io.BytesIO(val.new[0].content.tobytes()))))

def zoomfigure_func(b):
    fig1.set_dpi(100*b.new)

def zoomfigure_func1(b):
    fig2.set_dpi(100*b.new)
    
def resetzoom_func(b):
    floatslider.value = 1.5
    floatslider1.value = 1.5

slider1 = ipywidgets.FloatRangeSlider(value=config["intensity_clip"],min=0,max=1,description="Intensity clip",step=0.01)
slider3 = ipywidgets.IntRangeSlider(value=config["yc"],min=0,max=config["input_img"].shape[0],description="y-crop")
slider4 = ipywidgets.IntRangeSlider(value=config["xc"],min=0,max=config["input_img"].shape[1],description="x-crop")
check1 = ipywidgets.Checkbox(value=config["inv"],description="Inversion")
resetb = ipywidgets.Button(description="Reset")
fileup = ipywidgets.FileUpload()
floatslider = ipywidgets.FloatSlider(min=0.2,max=5,value=1.5,description="Plot zoom")
floatslider1 = ipywidgets.FloatSlider(min=0.2,max=5,value=1.5,description="Hist. zoom")
resetzoom = ipywidgets.Button(description="Reset zoom")
resetzoom.on_click(resetzoom_func)

resetb.on_click(widget_reset)
def widget_observe():
    slider1.observe(widget_change_intensity_clip,"value")
    slider3.observe(widget_change_y,"value")
    slider4.observe(widget_change_x,"value")
    check1.observe(widget_change_inv,"value")
    fileup.observe(updatefile,"value")
    floatslider.observe(zoomfigure_func,"value")
    floatslider1.observe(zoomfigure_func1,"value")
    

def widget_unobserve():
    slider1.unobserve(widget_change_intensity_clip,"value")
    slider3.unobserve(widget_change_y,"value")
    slider4.unobserve(widget_change_x,"value")
    check1.unobserve(widget_change_inv,"value")
    fileup.unobserve(updatefile,"value")
    floatslider.unobserve(zoomfigure_func,"value")
    floatslider1.unobserve(zoomfigure_func1,"value")

widget_observe()
wreset()
update_1(False)

imgeditor = ipywidgets.HBox([ipywidgets.VBox([ipywidgets.HBox([fileup,resetb]),slider4,slider3,check1,slider1,floatslider,floatslider1,resetzoom,fig2.canvas],layout=ipywidgets.Layout(width="30%",align_items="center")),fig1.canvas],layout=ipywidgets.Layout())
imgeditor

In [ ]:
config["edited_img"]

In [ ]:
%matplotlib widget
def predict_general_img(img,fn_model):
    #split image in 512x512 tiles
    sizey,sizex = img.shape
    lix = [((j*512,min((j+1)*512,sizey)),(i*512,min((i+1)*512,sizex))) for j in range(int(np.ceil(sizey/512))) for i in range(int(np.ceil(sizex/512)))]
    limgarray = []
    for ele in lix:
        pimg = img[ele[0][0]:ele[0][1],ele[1][0]:ele[1][1]]
        nimg = np.ones((512,512))
        nimg[:min(512,pimg.shape[0]),:min(512,pimg.shape[1])] = pimg
        limgarray.append(nimg)
    limgarray = np.array(limgarray)
    #Predict label
    lpredict = predict(limgarray,fn_model,batch_size=1,normalize_255=False)
    if "2023_model.keras" in fn_model:
        #Swap class index of 1 and 2, since for model 2023 the class indeces are (skyrmion:0, background:1, defects:2) and the functions are written for class indeces (skyrmion:0, defects:1, background:2)
        lpredict[lpredict==1] = 5
        lpredict[lpredict==2] = 1
        lpredict[lpredict==5] = 2
    #reconstruct full image from tiles
    pred_label = np.zeros((sizey,sizex),dtype=lpredict.dtype)
    for i,ele in enumerate(lix):
        pred_label[ele[0][0]:ele[0][1],ele[1][0]:ele[1][1]] = lpredict[i,:ele[0][1]-ele[0][0],:ele[1][1]-ele[1][0]]
    return pred_label

def trafo_channel_to_rgb(I):
    basis = np.array([[255,0,0],[0,255,0],[0,0,255],[255,255,0],[0,255,255]],dtype=np.uint8)
    return basis[I]

out = ipywidgets.Output()
with out:
    fig,ax = plt.subplots(ncols=2,dpi=100,figsize=(10,5))#,dpi=300)

fig.canvas.header_visible = False
ax[0].imshow(config["edited_img"],cmap="gray")
tmpimg=np.ones_like(config["edited_img"])
im_pred = ax[1].imshow(tmpimg)
ax[0].set_title("Kerr image")
ax[1].set_title("Predicted label")
ax[0].grid(False)
ax[1].grid(False)
fig.tight_layout()
prediction = {}
def zoomfigure_func(b):
    fig.set_dpi(100*b.new)
def resetzoom_func(b):
    floatslider.value = 1.5
def start_out():
    with out:
        out.clear_output()
        print("",end="\r")
def predict_func(b):
    start_out()
    with out:
        prediction["label"] = predict_general_img(config["edited_img"],modelfndic[dropdown_model.value])
        im_pred.set_data(trafo_channel_to_rgb(prediction["label"])) 
button_predict = ipywidgets.Button(description="Predict")
modeldic = {"Model 2023":0,"Model 2022":1,"Model 2022 inverse":2}
modelfndic = {0:'models/2023_model.keras',1:'models/2022_model.keras',2:'models/2022_model_inv.keras'}
dropdown_model = ipywidgets.Dropdown(options=modeldic)
floatslider = ipywidgets.FloatSlider(min=0.2,max=5,value=1.5,description="Plot zoom")
resetzoom = ipywidgets.Button(description="Reset zoom")
floatslider.observe(zoomfigure_func,"value")
resetzoom.on_click(resetzoom_func)
button_predict.on_click(predict_func)
out = ipywidgets.Output()
start_out()
ipywidgets.VBox([ipywidgets.HBox([floatslider,resetzoom]),ipywidgets.HBox([dropdown_model,button_predict]),out])

In [ ]:
prediction["label"]

## Skyrmion extraction & selection

In [ ]:
%matplotlib widget
def extract_skyrmions(pred_label):
    _,labels,stats,posl = cv2.connectedComponentsWithStats((pred_label==0).astype(np.uint8),cv2.CV_32S)
    #filter area
    ixl,areal,posl = np.arange(1,len(stats)),stats[1:,4],posl[1:]
    if selector["min_max_area"] != None:
        ixfilter = np.logical_and(areal>=selector["min_max_area"][0],areal<=selector["min_max_area"][1])
        ixl,areal,posl = ixl[ixfilter],areal[ixfilter],posl[ixfilter]
    ixsort = np.argsort(areal)[::-1]
    ixl,areal,posl = ixl[ixsort],areal[ixsort],posl[ixsort]
    ixlabel = np.zeros(np.max(labels)+1,dtype=np.int64)
    ixlabel[ixl] = np.arange(1,1+len(ixl))
    labels = ixlabel[labels]

    colmap = np.vstack((np.ones(3),0.87*np.random.rand(np.max(labels),3)))
    cont,_ = cv2.findContours((labels!=0).astype(np.uint8),cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    radiusl = np.sqrt(areal/np.pi)
    return labels,posl,areal,colmap,cont,radiusl

def update_selector(selector):
    labels,posl,areal,colmap,cont,radiusl = extract_skyrmions(prediction["label"])
    selector.update({"labels":labels,"posl":posl,"areal":areal,"colmap":colmap,"cont":cont,"radiusl":radiusl})
    
selector = {"min_max_area":None,"pred":prediction["label"]}
update_selector(selector)
selector["rbins"] = np.linspace(np.min(selector["radiusl"]),np.max(selector["radiusl"]),40)

fig = plt.figure(dpi=100,figsize=(10,6))
fig.canvas.header_visible = False
gs = fig.add_gridspec(2,2,height_ratios=[1,0.3])
ax = [fig.add_subplot(gs[0,0]),fig.add_subplot(gs[0,1]),fig.add_subplot(gs[1,:])]
im1 = ax[0].imshow(selector["colmap"][selector["labels"]])
im2 = ax[1].imshow(config["edited_img"],cmap="gray")

lcontobj =  []
for i in range(len(selector["cont"])):
    lx,ly = list(selector["cont"][i][:,0,0]),list(selector["cont"][i][:,0,1])
    lcontobj.append(ax[1].plot(lx+[lx[0]],ly+[ly[0]],color="r",lw=0.5)[0])
ax[1].set_title("Skyrmion contours")
ax[0].set_title("Single skyrmion mask")
ax[0].grid(False)
ax[1].grid(False)

_,_,hist = ax[2].hist(selector["radiusl"],bins=selector["rbins"])
meanrhist =  ax[2].axvline(np.mean(selector["radiusl"]),color="red",lw=3,label="Mean value")
ax[2].legend()
ax[2].set_xlabel(r"Radius $r=\sqrt{A/\pi}$ [pixel]")
ax[2].set_ylabel(r"Frequency")
ax[2].set_title("Skyrmion radius statistic")
fig.tight_layout()
def zoomfigure_func(b):
    fig.set_dpi(100*b.new)
def resetzoom_func(b):
    floatslider.value = 1.5
def select_func(b):
    while len(lcontobj)>0:
        obj = lcontobj.pop()
        obj.remove()
    selector["min_max_area"] = (selradius.value[0]**2*np.pi,selradius.value[1]**2*np.pi)
    update_selector(selector)
    for i in range(len(selector["cont"])):
        lx,ly = list(selector["cont"][i][:,0,0]),list(selector["cont"][i][:,0,1])
        lcontobj.append(ax[1].plot(lx+[lx[0]],ly+[ly[0]],color="r",lw=0.5)[0])
    im1.set_data(selector["colmap"][selector["labels"]])
    im2.set_data(config["edited_img"])
    
    nhist = np.histogram(selector["radiusl"],selector["rbins"],density=True)[0]
    for i,ele in enumerate(hist):
        ele.set_height(nhist[i])
    ax[2].set_ylim(0,np.max(nhist)*1.1)
    meanrhist.set_xdata([np.mean(selector["radiusl"]),np.mean(selector["radiusl"])])

floatslider = ipywidgets.FloatSlider(min=0.2,max=5,value=1.5,description="Plot zoom")
resetzoom = ipywidgets.Button(description="Reset zoom")
floatslider.observe(zoomfigure_func,"value")
resetzoom.on_click(resetzoom_func)

minr,maxr = np.min(selector["radiusl"]),np.max(selector["radiusl"])
selradius = ipywidgets.FloatRangeSlider(value=(minr,maxr),min=minr,max=maxr,readout_format=".1f",description="Radius range",step=0.01)
selradius.layout.width="40%"
selradius.observe(select_func,"value")

ipywidgets.VBox([ipywidgets.HBox([floatslider,resetzoom]),ipywidgets.HBox([selradius])])

## Saving extracted data in a pandas DataFrame & CSV file.

In [ ]:
dataframe = pd.DataFrame({"pos_x [pixel]":selector["posl"][:,0],"pos_y [pixel]":selector["posl"][:,1],"area [pixel*pixel]":selector["areal"]})
dataframe.to_csv("extracted_skyrmion_data_table.csv",sep=";")
dataframe

## Determination of average skyrmion-skyrmion distance
Baesd on the extracted data

In [ ]:
%matplotlib widget
def calculate_voronoi_delunay(posl):
    voronoi = scipy.spatial.Voronoi(posl)
    delaunay = scipy.spatial.Delaunay(posl)
    return voronoi,delaunay

def calculate_sky_sky_distance(voronoi,delaunay,posl,angle_lim):
    lcon,lcon1 = [],[]
    #selector out triangles with small angles (only occurs at the boundary of the image; therefore, they do not represent real skyrmion distances)
    for ele in delaunay.simplices:
        ok = True
        for i in range(len(ele)):
            v1,v2 = delaunay.points[ele[(i+1)%len(ele)]]-delaunay.points[ele[i]],delaunay.points[ele[(i-1)%len(ele)]]-delaunay.points[ele[i]]
            if not (np.pi/180*angle_lim[0]<=np.arccos(np.clip(np.dot(v1,v2)/np.sqrt(np.dot(v1,v1)*np.dot(v2,v2)),-1,1))<=np.pi/180*angle_lim[1]):
                ok = False
                break
        if ok:
            for i in range(len(ele)):
                a,b = ele[i],ele[(i+1)%len(ele)]
                lcon.append((min(a,b),max(a,b)))
        else:
            for i in range(len(ele)):
                a,b = ele[i],ele[(i+1)%len(ele)]
                lcon1.append((min(a,b),max(a,b)))
            
    lcon = list(set(lcon))
    lcon1 = list(set(lcon1)-set(lcon))
    distance = [np.linalg.norm(posl[a]-posl[b]) for a,b in lcon]
    return distance,lcon,lcon1

def update_dist(dist):
    if ("voronoi" not in dist) or ("delaunay" not in dist):
        dist["voronoi"],dist["delaunay"] = calculate_voronoi_delunay(dist["posl"])
    
    distance,lcon,lcon1 = calculate_sky_sky_distance(dist["voronoi"],dist["delaunay"],dist["posl"],dist["angle_lim"])
    dist.update({"distance":distance,"lcon":lcon,"lcon1":lcon1})

dist = {"posl":selector["posl"],"angle_lim":(0,180)}
update_dist(dist)

fig = plt.figure(dpi=100,figsize=(10,10))
fig.canvas.header_visible = False
gs = fig.add_gridspec(2,1,height_ratios=[2,0.5])
ax = [fig.add_subplot(gs[0]),fig.add_subplot(gs[1])]

#ax[0].plot(dist["posl"][:,0],dist["posl"][:,1],"ro",ms=0.3)

for ele in dist["voronoi"].regions:
    if len(ele)==0 or len(list(filter(lambda x:x==-1,ele)))>0: continue
    ax[0].plot([dist["voronoi"].vertices[i,0] for i in list(ele)+[ele[0]]],[dist["voronoi"].vertices[i,1] for i in list(ele)+[ele[0]]],lw=1.5,color="b")

lobj = []

for a,b in dist["lcon"]:
    lobj.append(ax[0].plot([dist["posl"][a,0],dist["posl"][b,0]],[dist["posl"][a,1],dist["posl"][b,1]],color="r",lw=1)[0])
    
for a,b in dist["lcon1"]:
    lobj.append(ax[0].plot([dist["posl"][a,0],dist["posl"][b,0]],[dist["posl"][a,1],dist["posl"][b,1]],color="g",lw=1)[0])

ax[0].set_xlim(0,config["edited_img"].shape[1])
ax[0].set_ylim(config["edited_img"].shape[0],0)
ax[0].imshow(config["edited_img"],cmap="gray",origin="lower")
ax[0].set_title("Delaunay triangulation result")
ax[0].grid(False)
yh,xh,hist = ax[1].hist(dist["distance"],color='#1f77b4',density=True)
histmean = ax[1].axvline(np.mean(dist["distance"]),color="red",lw=3,label="Periodicity")
lobj += [hist,histmean]
ax[1].set_ylim(0,np.max(yh)*1.2)
ax[1].set_xlim(np.min(xh),np.max(xh))
ax[1].set_xlabel(r"Skyrmion-Skyrmion distance [pixel]")
ax[1].legend()
ax[1].set_ylabel(r"Frequency")
ax[1].set_title("Skyrmion-Skyrmion statistic")
fig.tight_layout()

def zoomfigure_func(b):
    fig.set_dpi(100*b.new)
def resetzoom_func(b):
    floatslider.value = 1.5
def angle_func(update_val):
    while len(lobj)>0:
        obj = lobj.pop()
        obj.remove()

    dist["angle_lim"] = update_val.new
    update_dist(dist)
        
    for a,b in dist["lcon"]:
        lobj.append(ax[0].plot([dist["posl"][a,0],dist["posl"][b,0]],[dist["posl"][a,1],dist["posl"][b,1]],color="r",lw=1)[0])
    
    for a,b in dist["lcon1"]:
        lobj.append(ax[0].plot([dist["posl"][a,0],dist["posl"][b,0]],[dist["posl"][a,1],dist["posl"][b,1]],color="g",lw=1)[0])

    if len(dist["distance"])>0:
        yh,xh,histobj = ax[1].hist(dist["distance"],color='#1f77b4',density=True)
        histmean = ax[1].axvline(np.mean(dist["distance"]),color="red",lw=3,label="Periodicity")
        lobj.append(histobj)
        lobj.append(histmean)
        ax[1].set_ylim(0,np.max(yh)*1.2)
        ax[1].set_xlim(np.min(xh),np.max(xh))
        


floatslider = ipywidgets.FloatSlider(min=0.2,max=5,value=1.5,description="Plot zoom")
resetzoom = ipywidgets.Button(description="Reset zoom")
floatslider.observe(zoomfigure_func,"value")
resetzoom.on_click(resetzoom_func)

minr,maxr = np.min(selector["radiusl"]),np.max(selector["radiusl"])
selradius = ipywidgets.FloatRangeSlider(value=(minr,maxr),min=minr,max=maxr,readout_format=".1f",description="Radius range",step=0.01)
selradius.layout.width="40%"
selradius.observe(select_func,"value")

selradius = ipywidgets.FloatRangeSlider(value=(0,180),min=0,max=180,readout_format=".1f",description="Angle range",step=0.01)
selradius.layout.width="40%"
selradius.observe(angle_func,"value")

ipywidgets.VBox([ipywidgets.HBox([floatslider,resetzoom]),selradius])